# **Tell Me What You Want, I'll Tell You What You Like**

In this file, we try to use a better clustering approach (exploration then exploitation)

## DOcplex Python API installation

First, we install the docplex python API which we will use for modelling our problem and solving it. (Then, we will not have problem size error)

In [1]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex -q
        print("!pip install docplex -q...")
    else:
        !pip install --user docplex -q
        print("!pip install --user docplex -q...")
        
print("End of the intallation of python API...")

End of the intallation of python API...


## Main code

In [2]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from Solver import *
from User import *

ModuleNotFoundError: No module named 'Solver'

In [ ]:
#############################
### Essentials functions ###
#############################
#import functionMain du dossier parent
import sys
sys.path.append("..")
import FunctionMain as fm
import my_clustering as my_cl

In [ ]:
# ----------------- Parameters

# The file to consider
file = '../file_with_optimal_val/la04.txt'

# The number of solutions that we will have in the solver during the first iteration
k = 10

# The number of layer which is fixed
nb_layers = 5

# Variable which display the solution
display_sol = False

# Variable which display the start (in a vector)
display_start = False

# Variable which display the start (in a matrix)
display_matrix = False

# Time stop criterion 
tps_max = 300

# Number of iteration stop criterion
it_max = 15

# The number of solutions that we will have in the solver after the first iteration
k_k = 15

# List of different search type (for the solver)
list_search_type = ["DepthFirst", "Restart", "MultiPoint", "IterativeDiving"]

#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# --------- Interaction with the solver
data = []

# --------- Get essential information from the file
n, m, data, T_machine, T_duration, duration, optimalval = fm.get_data_from_file(file)

# --------- Call Solver constructor in Solver.py and create the variables of the model
model, solver, starts = fm.initialize_solver(data, n, m, duration)

# --------- Create the constraints of the model
model, variables = solver.create_constraints(model, n, m, optimalval, T_machine)

# ------------ Solve the model
print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)

# ------------ Display the result
fm.display_solution(msol, display_sol)
print("Model solved !")

# ---------------- Interaction with the user
print("\n--------Interaction with the user...---------")
print("\nCreating the user...")
user = User()
print("User created !")

#Get the variables of the model
variables = solver.get_variables()

list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)

# Vector of the start time of each task of each preference
starts = user.start_pref(n, m, display_start)

# Matrix of the start time of each task of each preference
matrix = user.matrix_pref(n, m, display_matrix)

# Testing the order of preferences and the differences between solutions
fm.test(n, m, user)

print("list layers : ",list_layers)


In [ ]:
####################################################################
#### Exploration: RECHERCHE DE NOUVELLES SOLUTIONS 
####################################################################


###  -------------- Iteration of the solver with the preferences
# Initialization of the counter of iteration
it = 1
# Initialization of the counter of time
tps = runtime
# Initialization of the list of the objective function (that we would like to plot later)
list_min_obj = [min(list_obj)]
# Initialization of the list of the objective function globally (that we would like to plot later)
list_min_obj_global = [min(list_obj)]
# Number of iterations (for exploration)
it_max_exploration = 0.75*it_max
# Time (for exploration)
tps_max_exploration = 0.75*tps_max
# Initialization of the stop criterion
criterion = (tps < tps_max_exploration) and (it < it_max_exploration) 

# ----------------- Add the preferences to the model
while criterion :
    print("\n--------Iteration {}---------".format(it))
    it += 1

    # --------- Call Solver constructor in Solver.py and create the variables of the model
    model, solver, starts = fm.initialize_solver(data, n, m, duration)

    # --------- Add the new constraints to the model (that solution must be different from the previous generated solutions)
    variables = fm.update_variables_new_constraint(n, m,  pref, model, solver)
    
    # # --------- Add the constraints considering the clustering
    list_rayon_layers, list_start_sol_layers, dict_sol_rayon  = my_cl.list_rayon_binaire_cluster(n, m, list_layers)
    list_rayon_layers_flatten = [item for sublist in list_rayon_layers[1:] for item in sublist]
    list_start_sol_layers_flatten = [item for sublist in list_start_sol_layers[1:] for item in sublist]
         
    for l in range(len(list_rayon_layers_flatten)):
        solver.add_constraint(model, my_cl.manhattan_binaire_distance_contrainte(list_start_sol_layers_flatten[l], [model.start_of(variables[i//m][i%m]) for i in range(n*m)]) > list_rayon_layers_flatten[l])
               
            
    # --------- Add the constraints considering the clustering
    model, variables = solver.create_constraints(model, n, m, optimalval, T_machine)

    # ------------ Solve the model
    print("\nSolving the model...")
    msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)

    print("The number of solutions generated is :",nb_solution)

    # Adding the objective value of each solution to a list
    list = []
    for sol in msol:
        list.append(user.objectiveFunction(sol) + user.objectiveFunctionRegularity(sol, n, m))
        # list.append(user.objectiveFunction(sol) * user.objectiveFunctionRegularity(sol, n, m))

    # ------------ Adding the min of objective function among the solutions generated to 
    # the list of objective function (for later display)
    list_min_obj.append(min(list))
    print("Objective function :", list_min_obj)

    # ------------ Display the result
    fm.display_solution(msol, display_sol)
    print("Model solved !")

    # ---------------- Interaction with the user
    list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)
    print("Il y a {} solution(s)".format(len(pref)))

    # ------------ Adding the min of objective function among all solutions generated to 
    # the list of objective function (for later display)
    list_min_obj_global.append(min(list_obj))
    print("Objective function global :", list_min_obj_global)

    # Vector of the start time of each task of each preference
    starts = user.start_pref(n, m, display_start)

    # Matrix of the start time of each task of each preference
    matrix = user.matrix_pref(n, m, display_matrix)

    # Testing the order of preferences and the differences between solutions
    fm.test(n, m, user)

#------------------ Stop criterion ------------------
    tps += runtime
    criterion = (tps < tps_max_exploration) and (it < it_max_exploration) 
    fm.stopCondition(it, it_max_exploration, tps, tps_max_exploration)
    
it_final_exploration = it

In [ ]:
# Plot the minimum objective among the solutions generated 

# import matplotlib.pyplot as plt
# print(list_min_obj)
# plt.plot([i for i in range(it)], list_min_obj, label='min obj', marker='o')
# plt.xlabel("Iteration")
# plt.ylabel("makespan")
# plt.title("Evolution of the makespan for generate solutions")
# plt.xticks(range(it))
# plt.legend()
# plt.show() 

# Print it_max exploration
print("\n--------End of the exploration---------")
print("The number of iterations is :", it)
print(list_min_obj_global)

In [ ]:
# Plot the minimum objective among the solutions generated so far

#print(list_min_obj_global)
# plt.plot([i for i in range(it)], list_min_obj_global, label='min obj', marker='o')
# plt.xlabel("Iteration")
# plt.ylabel("makespan")
# plt.title("Evolution of the best makespan so far")
# plt.xticks(range(it))
# plt.legend()
# plt.show() 

In [ ]:
####################################################################
#### Exploitation: RECHERCHE DE NOUVELLES SOLUTIONS 
####################################################################


###  -------------- Iteration of the solver with the preferences

# Number of solutions generated in the exploitation
k_k = 4
# Initialization of the stop criterion (exploitation)
criterion = (tps < tps_max) and (it < it_max) 

# ----------------- The loop for the problem (until a stop criterion is reached)
while criterion :
    print("\n--------Iteration {}---------".format(it))
    it += 1

    # --------- Call Solver constructor in Solver.py and create the variables of the model
    model, solver, starts = fm.initialize_solver(data, n, m, duration)

    # --------- Add the new constraints to the model (that solution must be different from the previous generated solutions)
    variables = fm.update_variables_new_constraint(n, m,  pref, model, solver)
    
    # # --------- Add the constraints considering the clustering
    list_rayon_layers, list_start_sol_layers, dict_sol_rayon  = my_cl.list_rayon_binaire_cluster(n, m, list_layers)
    list_cluster_layers = my_cl.my_clustering_binaire(n, m, list_rayon_layers, list_layers)
    list_centroides_layers = my_cl.centroides_clusters(n, m, list_cluster_layers)

    sum = 0
    # We iterate on the layers
    for i in range(len(list_centroides_layers)): 
        sum_temp = 0
        # We iterate on the clusters of each layer
        for j in range(len(list_centroides_layers[i])): 
            sum_temp += my_cl.manhattan_binaire_distance_contrainte(list_centroides_layers[i][j], [model.start_of(variables[i//m][i%m]) for i in range(n*m)])
        sum_temp *= len(list_centroides_layers)-i 
    sum += sum_temp
    model.add(minimize(sum))


    # list_rayon_layers_flatten = [item for sublist in list_rayon_layers[1:] for item in sublist]
    # list_start_sol_layers_flatten = [item for sublist in list_start_sol_layers[1:] for item in sublist]
         
    # for l in range(len(list_rayon_layers_flatten)):
    #     solver.add_constraint(model, my_cl.manhattan_binaire_distance_contrainte(list_start_sol_layers_flatten[l], [model.start_of(variables[i//m][i%m]) for i in range(n*m)]) > list_rayon_layers_flatten[l])
    
    # ------------ Solve the model
    print("\nSolving the model...")
    
    model, variables = solver.create_constraints(model, n, m, optimalval, T_machine)
    msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)
    print("The number of solutions generated is :",nb_solution)

    # Adding the objective value of each solution to a list
    list = []
    for sol in msol:
        list.append(user.objectiveFunction(sol)+user.objectiveFunctionRegularity(sol, n, m))
        # list.append(user.objectiveFunction(sol) * user.objectiveFunctionRegularity(sol, n, m))

    # ------------ Adding the min of objective function among the solutions generated to 
    # the list of objective function (for later display)
    list_min_obj.append(min(list))
    print("Objective function :", list_min_obj)

    # ------------ Display the result
    fm.display_solution(msol, display_sol)
    print("Model solved !")

    # ------------ Interaction with the user
    list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)
    print("Il y a {} solution(s)".format(len(pref)))

    # ------------ Adding the min of objective function among all solutions generated to 
    # the list of objective function (for later display)
    list_min_obj_global.append(min(list_obj))
    print("Objective function global :", list_min_obj_global)

    # Vector of the start time of each task of each preference
    starts = user.start_pref(n, m, display_start)

    # Matrix of the start time of each task of each preference
    matrix = user.matrix_pref(n, m, display_matrix)

    # Testing the order of preferences and the differences between solutions
    fm.test(n, m, user)

#------------------ Stop criterion ------------------
    tps += runtime
    criterion = (tps < tps_max) and (it < it_max) 
    fm.stopCondition(it, it_max, tps, tps_max)


In [ ]:
# Plot the minimum objective among the solutions generated

import matplotlib.pyplot as plt
print(list_min_obj)
plt.plot([i for i in range(it)], list_min_obj, label='min obj', marker='o')
plt.axvline(x=it_final_exploration, color='r', linestyle='--', label='it final exploration')
plt.xlabel("Iteration")
plt.ylabel("makespan")
plt.title("Evolution of the makespan for generate solutions")
plt.xticks(range(it))
plt.legend()
plt.show() 

In [ ]:
# Plot the minimum objective among all the solutions generated so far

print(list_min_obj_global)
plt.plot([i for i in range(it)], list_min_obj_global, label='min obj', marker='o')
plt.xlabel("Iteration")
plt.ylabel("makespan")
plt.title("Evolution of the best makespan so far")
plt.xticks(range(it))
plt.legend()
plt.show() 

In [ ]:
import main_cb as cb
list_min_obj, list_min_obj_global = cb.main_cb({}, '../file_with_optimal_val/la04.txt', "test0", 5, 20, 15, 100, 10, "plus")